<h1 style="color:blue; font-size:50px; font-family:Arial">Features Extraction</h1>


# Pourquoi?

L'extraction des caractéristiques des URL est importante pour un modèle de classification de phishing car elle permet de créer des signatures uniques pour les sites malveillants qui peuvent être utilisées pour détecter automatiquement les tentatives de phishing. Les caractéristiques couramment utilisées pour l'analyse de phishing incluent des éléments tels que le nom de domaine, les sous-domaines, les paramètres de l'URL, les numéros de port, les protocoles et les caractères spéciaux. En utilisant ces caractéristiques pour créer des signatures uniques, un modèle de classification peut être formé pour détecter automatiquement les sites malveillants et les ajouter à une liste noire. Cela permet de protéger les utilisateurs contre les tentatives de phishing en temps réel.


# Les données:

Un modèle de classification nécessite des données volumineuses pour deux raisons principales :

La complexité des données : Les modèles de classification sont utilisés pour reconnaître des patterns ou des tendances dans les données. Plus les données sont variées et complexes, plus il est difficile pour un modèle de les comprendre et de les classer correctement. Avoir un grand volume de données permet de capturer cette complexité et de mieux entraîner le modèle.

La précision du modèle : Plus le modèle est entraîné sur un grand volume de données, plus il est capable de généraliser correctement aux nouvelles données. Cela signifie qu'il est moins susceptible de commettre des erreurs lors de la classification de nouveaux exemples. Les données volumineuses permettent également de réduire la variance et l'erreur générale du modèle, augmentant ainsi la précision de la classification.

En somme, disposer d'un grand volume de données permet d'entraîner un modèle plus complexe et plus précis. Il est donc important de disposer d'un grand jeu de données pour entraîner efficacement un modèle de classification.

En revanche, par manque de ressources, et vu que l'objectif du présent projet est avant tout de présenter l'approche suivie pour identifier les attaques phishing, on a décidé de nous limiter à des data sets non volumineuses.

# <u> Phishing URLs Data source

Il n'y a pas de services gratuits d'API de phishing URL car la détection de phishing est un travail complexe qui nécessite des ressources importantes pour maintenir et mettre à jour régulièrement les bases de données de sites malveillants. Les entreprises qui offrent des services de détection de phishing facturent généralement leurs clients pour couvrir les coûts liés à ces activités de maintenance et de mise à jour. De plus, les API de phishing ont souvent des exigences de confidentialité et de sécurité élevées qui doivent être respectées pour protéger les données des utilisateurs. 

Pour Ce projet, on avait deux solutions: Utiliser une base des données csv gratuite, mais  l'utilisation d'une telle base de données gratuite pourra ne pas être aussi efficace que des services payants qui ont des moyens pour mettre à jour régulièrement les données et les entretenir. Alors on a decidé de recourir au site Openphish. 

## OpenPhish

OpenPhish est un service de détection de phishing automatisé qui utilise des techniques d'analyse de données et d'apprentissage automatique pour détecter les sites de phishing. Il utilise des données en temps réel pour identifier les sites malveillants et les ajouter à sa liste noire. Les utilisateurs peuvent accéder à cette liste noire via une API pour vérifier les URL et protéger leur entreprise contre les tentatives de phishing. OpenPhish est un service payant qui est utilisé par les entreprises pour protéger leurs employés et leurs données contre les menaces de phishing en ligne.

Vu nos ressources limitées, on s'est contenté de réaliser un simple web scrapping depuis la page d'accueil d'OpenPhish où les 500 derniers URLs considérés comme phising sont listés. 


In [1]:
import pandas as pd
import numpy as np
import ipaddress
import re
from urllib.parse import urlparse
from datetime import datetime
import requests
from bs4 import BeautifulSoup

In [2]:
url = "https://openphish.com/feed.txt"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

phishing_df= pd.DataFrame({"URL":str(soup).split()})

phishing_df=pd.DataFrame(phishing_df["URL"].apply(lambda x:x.rstrip(x[-1])))

phishing_df

,URL
0,http://www.solutionfun.info/landingpage/dd2638...
1,http://36560022.com
2,http://www.panicasa.com
3,http://5467wa.ihostfull.com
4,http://nodesdatas.online
...,...
495,https://welltododearl.polarcita36.repl.co
496,http://binaryresistor.com
497,https://oidc.idp.elogin.e-access-itservices.co...
498,http://dontforgettoeat-foodforthought.com


In [3]:
a="a;lamlm/"
a.rstrip(a[-1])

'a;lamlm'

# <u> Legitimate URLs Data source

# Moz Top 500

Moz Top 500 est une liste des sites les plus populaires sur Internet, classés par leur autorité de domaine (DA) de Moz. Le DA est un score de 1 à 100 qui mesure la qualité et la quantité des liens pointant vers un site. Plus le DA d'un site est élevé, plus il est considéré comme fiable et de qualité par les moteurs de recherche. La liste Moz Top 500 est mise à jour régulièrement et est basée sur les données de l'outil de référencement de Moz, Open Site Explorer. 

Cette liste est utilisée pour fournir des URLs légitimes pour ce projet de classification de phishing URLs. En utilisant des URLs de sites de haute qualité et de confiance, classés par leur autorité de domaine, on peut entraîner un modèle de classification pour reconnaître les URLs légitimes et les distinguer des URLs de phishing.

In [4]:
leg_df=pd.read_csv("top500Domains.csv",usecols=["Root Domain"])
leg_df.columns=["URL"]
leg_df["URL"]="https://"+leg_df["URL"] #juste pour la propre execution de feature 8
leg_df

,URL
0,https://www.blogger.com
1,https://www.google.com
2,https://youtube.com
3,https://linkedin.com
4,https://support.google.com
...,...
495,https://merriam-webster.com
496,https://public-api.wordpress.com
497,https://techradar.com
498,https://greenpeace.org


### Featue 1: Le nom du domaine de l' URL

In [5]:
import tldextract
def extract_domain_name(url):
    extracted = tldextract.extract(url)
    return extracted.domain

### Featue 2: La présence du IP adress  dans le URL

On vérifie la présence de l'adresse IP dans l'URL. Les URLs peuvent avoir une adresse IP au lieu d'un nom de domaine. Si c'est le cas, nous pouvons être sûrs que quelqu'un essaie de voler des informations personnelles avec cette URL.

Si la partie domaine de l'URL contient une adresse IP, la valeur attribuée à Ce feature est 1 (Phishing) et 0 (legitimate) si non.

In [6]:
def ip_ds_URL(url):
    try:
        ip_address = re.search(r'[0-9]+(?:\.[0-9]+){3}', url).group()
        ip = 1
    except:
        ip = 0
    return ip


### Featue 3 : La présence du "@"  dans le URL

L'utilisation du symbole "@" dans l'URL conduit le navigateur à ignorer tout ce qui précède le symbole "@". Ainsi, de nombreux attaquants l'utilisent pour tromper les utilisateurs.

Si l'URL contient le symbole '@', la valeur attribuée à Ce feature est 1 (Phishing) et 0 (legitimate) si non.

In [7]:
def at_ds_URL(url):
    if "@" in url:
        at = 1    
    else:
        at = 0    
    return at

### Feature 4: longueur de l' URL

D'abord on calcule les moyennes des longueurs des phishing URLs et des Legitimate URLs.

Si la longueur de l'URL est plus proche à la moyenne des phishing URLs, la valeur attribuée à Ce feature est 1 (Phishing) et 0 (legitimate) si non.

In [8]:
# la moyenne des longueurs des phishing URLs.
m_phish= int(np.array(phishing_df['URL'].apply(lambda x: len(x))).mean())
# la moyenne des longueurs des Legitimate URLs
m_leg= int(np.array(leg_df['URL'].apply(lambda x: len(x))).mean())

def long_URL(url):
    if (abs(len(url)-m_phish) > abs(len(url)-m_leg)):
        return 0            
    else:
        return 1            


### Feature 5: Profondeur de l' URL

La profondeur d'un site correspond au nombre de clics entre une page donnée et la page d'accueil.

Ce feature calcule le nombre de sous-pages dans l'URL donnée en fonction du '/'.

In [9]:
def prof_URL(url):
    s = urlparse(url).path.split('/')
    depth = 0
    for j in range(len(s)):
        if len(s[j]) != 0:
            depth = depth+1
    return depth

### Feature 6: Nombre de caractères spéciaux dans l' URL

In [10]:
def nb_of_special_caract(url):
    special_characters = re.findall(r"[^a-zA-Z0-9./]", url)
    return len(special_characters)

### Feature 7:  Redirection "//" dans l' URL

On vérifie la présence de "//" dans l'URL. L'existence de "//" dans le domaine de l'URL signifie que l'utilisateur sera redirigé vers un autre site Web.

Si le "//" est n'importe où dans l'URL ( sauf apres http ou https) , la valeur attribuée à ce feature est 1 (phishing) et 0 (legitimate) si non.

In [11]:
def redirection(url):
    pos = url.rfind('//')
    if pos >= 7:  # après http:// (7 caractère)
        if pos >= 8: # après https:// (8 caractère)
            return 1
        else:
            return 0
    else:
        return 0

### Feature 8: La présence du "http" ou "https" dans le domaine de l' URL

On vérifie la présence de "http/https" dans la partie domaine de l'URL. Les phishers peuvent les ajouter à la partie domaine d'une URL afin de tromper les utilisateurs.

Si l'URL contient "http/https" dans son domaine, la valeur attribuée à ce feature est 1 (phishing) et 0 (legitimate) si non.

In [12]:
def http_ds_domaine(url):
    domain = urlparse(url).netloc
    if ('http' or 'https') in domain:
        return 1
    else:
        return 0

### Feature 9: L'utilisation des réducteurs d'URL 

Un réducteur d'URL est un outil qui permet de raccourcir un lien. De nombreux outils en ligne proposent aux utilisateurs de réduire automatiquement leurs liens trop longs et sont largement utilisé dans les phsihing attaques.

In [13]:
# les services les plus utiliser
red_serv = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"

In [14]:
def red_URL(url):
    match=re.search(red_serv,url)
    if match:
        return 1
    else:
        return 0

### Le service Whois

Whois est un service de recherche fourni par les registres Internet permettant d'obtenir des informations sur une adresse IP ou un nom de domaine. 

On va utiliser cette base des données afin d'extraire des features addtionnelles.

In [15]:
#!pip install python-whois
import whois

### Feature 10: Si l'URL existe dans la base des données whois

c'est implémenté automatiquement dans la fonction des Feature_extraction.

si il n'existe pas,  la valeur attribuée à ce feature est 1 (phishing) et 0 (legitimate) si non.


### Feature 11:Période de validité du domaine

Ce Feature est calculé en utilisant la base de données WHOIS. La plupart des sites Web de phishing ont une courte période de validité. On va calculer la période moyenne des phishing urls, de même pour des legitimate urls.

Si la Période de validité de l'URL est plus proche de la moyenne des phishing urls, la valeur attribuée à ce feature est 1 (phishing) et 0 (legitimate) si non.

##### On commence en calculant  la période de validité moyenne des phishing urls. 
<b>NB:</b> La time complexity de ce calcul est élévée. Dès lors, on se contente de le réaliser une seule fois. Au cas où on décide de changer les données, on prend une valeur approximative de celle qu'on a déjà trouvé sans avoir à refaire le calcul.

In [16]:
#phish_sample=phishing_df.sample(500).reset_index(drop=True)
pv_of_phsishing_urls=[]

for i in range (len(phishing_df)):
    url=phishing_df['URL'][i]
    try:
        domain_name=whois.whois(urlparse(url).netloc)
        creation_date = domain_name.creation_date
        expiration_date = domain_name.expiration_date
        if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
            try:
                creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
                expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
            except:
                continue
                
        elif ((type(expiration_date) is list) and (type(creation_date) is list)):
            creation_date = creation_date[0]
            expiration_date = expiration_date[0] 
            
        elif (type(expiration_date) is list):
                expiration_date = expiration_date[0]
                
        elif (type(creation_date) is list):
                creation_date = creation_date[0]        
        
        pv_of_phsishing_urls.append((expiration_date-creation_date).days)
    except:
        continue
        
mean_pv_phish=sum(pv_of_phsishing_urls)/len(pv_of_phsishing_urls) 
mean_pv_phish

Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001]

1890.5974842767296

##### puis on calcule la période de validité moyenne des legitimate urls
<b>NB:</b> La time complexity de ce calcul est élévée. Dès lors, on se contente de le réaliser une seule fois. Au cas où on décide de changer les données, on prend une valeur approximative de celle qu'on a déjà trouvé sans avoir à refaire le calcul.

In [17]:
#leg_sample=leg_df.sample(500).reset_index(drop=True)
pv_of_leg_urls=[]
for i in range (len(leg_df)):
    url = leg_df['URL'][i]
    try:
        domain_name=whois.whois(urlparse(url).netloc)
        creation_date = domain_name.creation_date
        expiration_date = domain_name.expiration_date
        if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
            try:
                creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
                expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
            except:
                continue
                
        elif ((type(expiration_date) is list) and (type(creation_date) is list)):
            creation_date = creation_date[0]
            expiration_date = expiration_date[0] 
            
        elif (type(expiration_date) is list):
                expiration_date = expiration_date[0]
                
        elif (type(creation_date) is list):
                creation_date = creation_date[0]
                
        pv_of_leg_urls.append((expiration_date-creation_date).days)
    except:
        continue
        
mean_pv_leg=sum(pv_of_leg_urls)/len(pv_of_leg_urls) 
mean_pv_leg

Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed


8960.993119266055

In [18]:
def periode_vie_url(domain_name):
    try:
        creation_date = domain_name.creation_date
        expiration_date = domain_name.expiration_date
        
        if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
            try:
                creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
                expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
            except:
                return 1
                
        elif ((type(expiration_date) is list) and (type(creation_date) is list)):
            creation_date = creation_date[0]
            expiration_date = expiration_date[0] 
            
        elif (type(expiration_date) is list):
                expiration_date = expiration_date[0]
                
        elif (type(creation_date) is list):
                creation_date = creation_date[0]
        
        
        pvofdomain=(expiration_date-creation_date).days
        
        if abs(pvofdomain-mean_pv_phish)<abs(pvofdomain-mean_pv_leg):
            return 1
        else:
            return 0
        
    except:
        return 1

### Feature 12:l'âge du domaine

Ce Feature est calculé en utilisant la base de données WHOIS. La plupart des sites Web de phishing vivent pendant une courte période. On va calculer l'âge moyen des phishing urls, de même pour des legitimate urls.

Si l'âge du domaine est plus proche de l'âge moyen des phishing urls, la valeur attribuée à ce feature est 1 (phishing) et 0 (legitimate) si non.

###### On calcule de l'âge moyen d'un échantillon des phishing urls
<b>NB:</b> La time complexity de ce calcul est élévée. Dès lors, on se contente de le réaliser une seule fois. Au cas où on décide de changer les données, on prend une valeur approximative de celle qu'on a déjà trouvé sans avoir à refaire le calcul.

In [19]:
#phish_sample=phishing_df.sample(500).reset_index(drop=True)
age_of_phsishing_urls=[]

for i in range (len(phishing_df)):
    url=phishing_df['URL'][i]
    try:
        domain_name=whois.whois(urlparse(url).netloc)
        creation_date = domain_name.creation_date
        if isinstance(creation_date,str):
            try:
                creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
            except:
                continue
        elif (type(creation_date) is list):
                creation_date = creation_date[0]        
        
        age_of_phsishing_urls.append((datetime.now()-creation_date).days)
    except:
        continue
        
mean_age_phish=sum(age_of_phsishing_urls)/len(age_of_phsishing_urls) 
mean_age_phish

Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001]

1523.191222570533

###### On calcule de l'âge moyen d'un échantillon des legitimate urls
<b>NB:</b> La time complexity de ce calcul est élévée. Dès lors, on se contente de le réaliser une seule fois. Au cas où on décide de changer les données, on prend une valeur approximative de celle qu'on a déjà trouvé sans avoir à refaire le calcul.

In [20]:
#leg_sample=leg_df.sample(500).reset_index(drop=True)
age_of_leg_urls=[]
for i in range (len(leg_df)):
    url = leg_df['URL'][i]
    try:
        domain_name=whois.whois(urlparse(url).netloc)
        creation_date = domain_name.creation_date
        if isinstance(creation_date,str):
            try:
                creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
            except:
                continue
        elif (type(creation_date) is list):
                creation_date = creation_date[0]        
        
        age_of_leg_urls.append((datetime.now()-creation_date).days)
    except:
        continue
        
mean_age_leg=sum(age_of_leg_urls)/len(age_of_leg_urls) 
mean_age_leg

Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed


8214.798185941043

In [21]:
def age_url(domain_name):
    try:
        creation_date = domain_name.creation_date
        expiration_date = domain_name.expiration_date
        if isinstance(creation_date,str):
            try:
                creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
            except:
                return 1
        elif (type(creation_date) is list):
                creation_date = creation_date[0]
                
        ageofdomain=(datetime.now()-creation_date).days
        
        if abs(ageofdomain-mean_age_phish)<abs(ageofdomain-mean_age_leg):
            return 1
        else:
            return 0
    except:
        return 1

### Feature 13: IFrame Redirection

Les phishers utilisent les iframes HTML pour intégrer des pages web malveillantes dans des pages web légitimes dans le but de voler des informations personnelles ou financières des utilisateurs. Les iframes permettent aux attaquants de masquer les pages malveillantes en les intégrant dans des pages web légitimes, ce qui rend plus difficile pour les utilisateurs de détecter les tentatives de phishing.



In [22]:
def iframe(response):
    if response == "":
        return 1
    else:
        if re.findall(r"[<iframe>|<frameBorder>]", response.text):
            return 0
        else:
            return 1

### Feature 14: Website Forwarding

Le Website Forwarding, également appelé redirection de site web, est une fonctionnalité qui permet de rediriger automatiquement les utilisateurs d'un site web vers une autre URL lorsqu'ils accèdent au site d'origine.

Les phishers utilisent souvent le Website Forwarding pour rediriger les utilisateurs vers des sites web de phishing qui ressemblent à des sites web légitimes.

Ce feature va juste compter les redictions de l'URL.

<b> NB: </b> la différence entre ce feature est le feature 7 est que le forwarding de site web est une fonctionnalité qui permet de rediriger les utilisateurs vers une autre URL alors que "//" est une syntaxe utilisée dans les URLs pour spécifier un protocole relatif.

In [23]:
def forwarding(response):
    if response == "":
        return 1
    else:
        return len(response.history)

In [24]:
#Function to extract features
def featureExtraction(url,label):

    features = []
    features.append(extract_domain_name(url))
    features.append(ip_ds_URL(url))
    features.append(at_ds_URL(url))
    features.append(long_URL(url))
    features.append(prof_URL(url))
    features.append(nb_of_special_caract(url))
    features.append(redirection(url))
    features.append(http_ds_domaine(url))
    features.append(red_URL(url))
    
    wis = 0
    try:
        domain_name = whois.whois(urlparse(url).netloc)
    except:
        wis = 1
        
    features.append(wis)
    features.append(1 if wis == 1 else periode_vie_url(domain_name))
    features.append(1 if wis == 1 else age_url(domain_name))
    
    try:
        response = requests.get(url,timeout=20)
    except:
        response = ""
    
    features.append(iframe(response))
    features.append(forwarding(response))
    
    features.append(label)
    
    return features

# L'extraction des features

## 1. Phishing URLs


In [25]:
phish_features = []
label = 1

for i in range(0, 500):
    
    url = phishing_df['URL'][i]
    phish_features.append(featureExtraction(url,label))

Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001]

In [26]:
feature_names=["Nom_de_Domaine","have_IP","have_at","longueur","profondeur","nb_de_caractere_spec",
               "redirection","http_ds_domaine","red_URL","whois","periode_de_vie","age_url","iframe","forwarding","label"]
phishing = pd.DataFrame(phish_features, columns= feature_names)
phishing.head()

,Nom_de_Domaine,have_IP,have_at,longueur,profondeur,nb_de_caractere_spec,redirection,http_ds_domaine,red_URL,whois,periode_de_vie,age_url,iframe,forwarding,label
0,solutionfun,0,0,1,3,6,0,0,0,0,1,1,0,1,1
1,36560022,0,0,0,0,1,0,0,0,0,1,1,1,1,1
2,panicasa,0,0,0,0,1,0,0,0,0,1,1,0,0,1
3,ihostfull,0,0,0,0,1,0,0,0,0,1,1,0,0,1
4,nodesdatas,0,0,0,0,1,0,0,0,0,1,1,1,1,1


## 2. legitimate URLs

In [27]:
legi_features = []
label = 0

for i in range(500):

    url = leg_df['URL'][i]
    legi_features.append(featureExtraction(url,label))

Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed


In [28]:
feature_names=["Nom_de_Domaine","have_IP","have_at","longueur","profondeur","nb_de_caractere_spec",
               "redirection","http_ds_domaine","red_URL","whois","periode_de_vie","age_url","iframe","forwarding","label"]
legitimate = pd.DataFrame(legi_features, columns= feature_names)
legitimate.head()

,Nom_de_Domaine,have_IP,have_at,longueur,profondeur,nb_de_caractere_spec,redirection,http_ds_domaine,red_URL,whois,periode_de_vie,age_url,iframe,forwarding,label
0,blogger,0,0,0,0,1,0,0,0,0,0,0,0,4,0
1,google,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,youtube,0,0,0,0,1,0,0,0,0,0,0,0,1,0
3,linkedin,0,0,0,0,1,0,0,0,0,0,0,0,1,0
4,google,0,0,0,0,1,0,0,0,0,0,0,0,0,0


#  Final Dataset

In [29]:
#Concaténer les deux dataframes 
urldata = pd.concat([legitimate, phishing]).reset_index(drop=True)
urldata.head()

,Nom_de_Domaine,have_IP,have_at,longueur,profondeur,nb_de_caractere_spec,redirection,http_ds_domaine,red_URL,whois,periode_de_vie,age_url,iframe,forwarding,label
0,blogger,0,0,0,0,1,0,0,0,0,0,0,0,4,0
1,google,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,youtube,0,0,0,0,1,0,0,0,0,0,0,0,1,0
3,linkedin,0,0,0,0,1,0,0,0,0,0,0,0,1,0
4,google,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [30]:
urldata.to_csv('urldata.csv', index=False)

# Conclusion
L'objectif de ce notebook est atteint. Nous avons finalement extrait 15 features pour 1000 URL dont 500 URL sont de phishing et 500 URL sont légitimes.

# Références

http://eprints.hud.ac.uk/id/eprint/24330/6/MohammadPhishing14July2015.pdf
